In [5]:
import os
import pandas as pd
import json
import shutil
import random
import numpy as np

In [6]:
def create_folder(path):
    try:
        os.mkdir(path)
    except:
        pass

In [14]:
# save all images in a folder which were used for training
def saveAllImagesInaFolder(input_path,out_path):
    all_images = os.listdir(input_path)
    for img in all_images:
        src = input_path+'/'+img
        dst = out_path+'/'+img
        shutil.copy(src,dst)

# to make sure all the images mask version also available we will choose image name from mask folder only
# keep only colored images for testing 
def getFileNameOfMaskedImages(img_path,img_details_path,colored_images,country):
    ## all the valid images
    df_org = pd.read_csv(img_details_path)

    df_color = pd.read_csv(colored_images,header = 0)
    df_color = df_color[df_color['status'] == 'OK']
    ## Keep only images which are colored images
    df_color = df_color.reset_index(drop=True)
    for i in range(len(df_color)):
        df_color.at[i,'fileName'] = df_color['fileName'][i].split('.')[0]
    df_color = df_color.rename(columns={'fileName': 'filename'})
    # merge original and color DataFrame
    df_org = pd.merge(df_org[['filename','gender']],df_color[['filename']],on = 'filename')
    
    ## all the masked images
    list_all = os.listdir(img_path)
    for i in range(len(list_all)):
        list_all[i] = list_all[i].split('.')[0].split('_N95')[0]
    df_all = pd.DataFrame(list_all, columns=['filename'])
    df = pd.merge(df_org,df_all,on = 'filename')
    df = df[['filename','gender']]
    return df
    
    
# create test dataset (take 60 images from each country 20 female and 40 male, total = 480 images)
def createTestData(df,img_path_country,training_img_path,out_path,country):
    cnt_male = 40
    cnt_female = 20
    
    train_list = os.listdir(training_img_path)
    for i in range(len(train_list)):
        train_list[i] = train_list[i].split('.')[0].split('_N95')[0]
    ## Images which were used for training
    df = df[~df['filename'].isin(train_list)]
    
    df_male = df[df['gender'] == 'M']
    df_male = df_male.reset_index()
    df_male = df_male.drop('index',axis = 1)
    df_female = df[df['gender'] == 'F']
    df_female = df_female.reset_index()
    df_female = df_female.drop('index',axis = 1)
    print("male : ",len(df_male))
    print("female : ",len(df_female))
#     return

    male_ind =  random.sample(range(0, len(df_male)), cnt_male)
    female_ind =  random.sample(range(0, len(df_female)), cnt_female)
    
    ## Pick cnt number of male and female images
    for i in male_ind:
        filename = df_male['filename'][i]+'.jpg'
        src = img_path_country+"/"+filename
        dst = out_path+'/'+df_male['filename'][i]+'_'+country+'.jpg'
        shutil.copy(src,dst)

    for i in female_ind:
        filename = df_female['filename'][i]+'.jpg'
        src = img_path_country+"/"+filename
        dst = out_path+'/'+df_female['filename'][i]+'_'+country+'.jpg'
        shutil.copy(src,dst)
            
    print(len(male_ind))
    print(len(female_ind))

In [13]:
image_type = 'original' # original, greyish, rgb, spread

train_data_path = "./../deepface-few-shot-v2/"+image_type
data_folders = []
for run in [1,2,3]:
    for num in [1,2]:
        folder_name = "run_"+str(run)+"/"+str(num)+"_shot"
        data_folders.append(folder_name)

# save all training data into 1 folder
out_train_path = 'tarining_'+image_type
create_folder(out_train_path)
for folder_name in data_folders:
    input_path_female = train_data_path+'/'+folder_name+'/female'
    saveAllImagesInaFolder(input_path_female,out_train_path)
    input_path_male = train_data_path+'/'+folder_name+'/male'
    saveAllImagesInaFolder(input_path_male,out_train_path)

In [15]:
all_img_path = './../Players Dataset'
img_details_path = './../image_details'
colored_images = "./../correction/status_csv_files"
out_test_path = 'test_data_trained_on_'+image_type
create_folder(out_test_path)
training_img_path = out_train_path
countries = os.listdir(all_img_path)
for country in countries:
    img_path = all_img_path+'/'+country+'/Original/masked'
    df = getFileNameOfMaskedImages(img_path,img_details_path+'/'+country+'.csv',colored_images+'/'+country+'.csv',country)
    img_path_country = all_img_path+'/'+country+'/Original/normal'
    print(country)
    createTestData(df,img_path_country,training_img_path,out_test_path,country)

Australia
male :  642
female :  149
40
20
Bangladesh
male :  310
female :  31
40
20
England
male :  908
female :  102
40
20
India
male :  1957
female :  178
40
20
New Zealand
male :  304
female :  212
40
20
Pakistan
male :  426
female :  69
40
20
South Africa
male :  413
female :  49
40
20
West Indies
male :  482
female :  54
40
20
